<center><h1>Predicting the Age of Abalones Using Physical Characteristics</h1></center>

<center><h2>Introduction</h2></center>

Abalones are marine snails that have a spiral structure, and are characterized by several open respiratory pores in 
a row near the shell's outer edge. Scientific studies on abalones often require knowing the age of the abalones. However, judging the age of abalones is a complicated and lengthy process. It requires counting the number of layers that form the abalone's shell, also known as the "rings". In order to do this, a sample of a shell is taken, stained and then the number of rings is counted under a microscope. However, some studies have shown that the judging of the age of abalones can be done through the observation of their physical characteristics.

**The question we therefore aim to answer through this data science project is this: How can the age of abalones be predicted using only physical characteristics and if so, how accurate is the prediction?**

<span>
    <img src="images/live_abalone.jpg" alt="live abalone" style="width: 500px;"/>
    <img src="images/abalone_shell.jpg" alt="abalone shell" style="width: 500px;"/>
</span>

<br/>

<center><h2>Data Used</h2></center>

We found a dataset which provides data on the physical characteristics of 4177 abalones on the UCI Machine Learning Repository. The variables provided in the dataset include - Sex, Length, Diameter, Height, Whole Weight, Shucked Weight, Viscera Weight, Shell Weight and Rings. More explaination about what these individually mean and the data type of the individual variables is given below:

![variable information](images/variables.png)

<center><h2>Data Wrangling</h2></center>

Firstly, importing all the modules needed in this project and doing some inital setup:

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import scipy.cluster
import sklearn.linear_model
import sklearn.model_selection
import plotly.offline as plt
import plotly.graph_objs as go
from IPython.core.display import display, HTML

#In order to render LaTeX in plotly graphs properly
display(HTML(
    '<script>'
        'var waitForPlotly = setInterval( function() {'
            'if( typeof(window.Plotly) !== "undefined" ){'
                'MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });'
                'MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);'
                'clearInterval(waitForPlotly);'
            '}}, 250 );'
    '</script>'
))

plt.init_notebook_mode()

Secondly, importing the data into a pandas dataframe object:

In [2]:
df = pd.read_csv("abalone.csv", names=["sex", "length", "diameter", "height", "wholeweight", "shuckedweight", "visceraweight", "shellweight", "rings"])
df.head()

,sex,length,diameter,height,wholeweight,shuckedweight,visceraweight,shellweight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


We immediately decided to remove the 'sex' variable from the dataset as it is a categorical variable and is tough to work with. We understand the implications of this (we might be ignoring an important variable that can have a large effect on the prediction of the age of the abalone), however, it is simply too complicated to work with.

In [3]:
df = df.drop("sex", axis=1)
df.head()

,length,diameter,height,wholeweight,shuckedweight,visceraweight,shellweight,rings
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


Then we checked for missing data (fortunately there is none):

In [4]:
df.isnull().sum()

length           0
diameter         0
height           0
wholeweight      0
shuckedweight    0
visceraweight    0
shellweight      0
rings            0
dtype: int64

<center><h2>Initial Analysis</h2></center>

We then decided to do some intial analysis to get a feel of the data, through the use of basic plots to help us to immediately identify any obvious trends visually.

Plot of distribution of "rings" variable in dataset:

In [5]:
mean = np.mean(df.rings)
mode = np.array(scipy.stats.mode(df.rings))[0][0]
median = np.median(df.rings)

hist = go.Histogram(name="Rings", x=df.rings)
meanline = go.Scatter(name="Mean", x=[mean for i in range(700)], y=[i for i in range(700)])
modeline = go.Scatter(name="Mode", x=[mode for i in range(750)], y=[i for i in range(750)])
medianline = go.Scatter(name="Median", x=[median for i in range(700)], y=[i for i in range(700)])
layout = go.Layout(title="Distribution of 'rings' Variable in Dataset", xaxis=dict(title="Number of Rings"), yaxis=dict(title="Frequency"))
fig = go.Figure(data = [hist, meanline, modeline, medianline], layout=layout)
plt.iplot(fig)

Plot of distribution of "shellweight" variable in the dataset:

In [6]:
mean = np.mean(df.shellweight)
mode = np.array(scipy.stats.mode(df.shellweight))[0][0]
median = np.median(df.shellweight)

hist = go.Histogram(name="Shell Weight", x=df.shellweight)
meanline = go.Scatter(name="Mean", x=[mean for i in range(140)], y=[i for i in range(140)])
modeline = go.Scatter(name="Mode", x=[mode for i in range(140)], y=[i for i in range(140)])
medianline = go.Scatter(name="Median", x=[median for i in range(140)], y=[i for i in range(140)])
layout = go.Layout(title="Distribution of 'shellweight' Variable in Dataset", xaxis=dict(title="Shell Weight"), yaxis=dict(title="Frequency"))
fig = go.Figure(data = [hist, meanline, modeline, medianline], layout=layout)
plt.iplot(fig)

<center><h2>Hypothesis Testing</h2></center>

### Hypothesis Test 1

In this hypothesis test, we are testing the null hypothesis that the population mean of the top 10% of the shell weights sorted according to the number of rings is the same as the population mean of the bottom 10% of the shell weights sorted according to the number of rings (they are statistically the same and any differences are just due to random chance) and the alternative hypothesis that the population means are different (they are statistically different). A significance level of 0.05 was chosen as there are no large risks from Type $I$ and $II$ errors.

$H_0$: The population mean of the top 10% of the shell weights sorted according to the number of rings is the same as the population mean of the bottom 10% of the shell weights sorted according to the number of rings (they are statistically the same and any differences are just due to random chance).

$H_1$: The population mean of the top 10% of the shell weights sorted according to the number of rings is the different from the population mean of the bottom 10% of the shell weights sorted according to the number of rings (they are statistically different and differences are not due to random chance).

$\alpha = 0.05$

The test will be two-sided, as we want to evaluate the probabilities of the mean of the top 10% being both higher or lower than the bottom 10%.

First the shell weights were sorted according to rings and the first 20 values were printed:

In [7]:
sorted_data = df.set_index("rings").sort_index()
shellweight_sorted = sorted_data["shellweight"]
print(shellweight_sorted[:20])
shellweight_sorted = shellweight_sorted.values

rings
1    0.0015
2    0.0050
3    0.0100
3    0.0125
3    0.0050
3    0.0110
3    0.0170
3    0.0050
3    0.0140
3    0.0100
3    0.0030
3    0.0090
3    0.0040
3    0.0105
3    0.0050
3    0.0140
3    0.0040
4    0.0130
4    0.0110
4    0.0430
Name: shellweight, dtype: float64


Then the sorted data was split up into the top 10% and bottom 10%:

In [8]:
obs = len(shellweight_sorted)
percent = int(obs/10)

top = shellweight_sorted[obs-percent:]
bottom = shellweight_sorted[:percent+1]

After which the Mann-Whitney U-test was conducted (since the data is clearly non-normal from the histogram above):

In [9]:
ustat, pval = scipy.stats.mannwhitneyu(top, bottom, alternative="two-sided")
print("U-statistic: " + str(ustat))
print("p-value: " + str(pval))

U-statistic: 173042.5
p-value: 3.85250296282e-134


Since the two sided p-value is lesser than the significance level, the null hypothesis is rejected in favour of the alternative hypothesis that the population means of the top and bottom 10% of shell weights sorted according to number of rings are statistically different.

### Hypothesis Test 2

In this hypothesis test, we are testing the null hypothesis that the population mean of the first 10% of the shell weights is the same as the population mean of the last 10% of the population weights (they are statistically the same and any differences are just due to random chance) and the alternative hypothesis that the population means are different (they are statistically different). A significance level of 0.05 was chosen as there are no large risks from Type $I$ and $II$ errors.

$H_0$: The population mean of the first 10% of the shell weights is the same as the population mean of the last 10% of the population weights (they are statistically the same and any differences are just due to random chance).

$H_1$: The population means are different (they are statistically different).

$\alpha$ = 0.05

The test will be two-sided, as we want to evaluate the probabilities of the mean of the first 10% being both higher or lower than the last 10%.

First the data was split up into the required samples:

In [10]:
obs = len(df.shellweight)
percent = int(obs/10)

last = df.shellweight.values[obs-percent:]
first = df.shellweight.values[:percent+1]

Then the Mann-Whitney U-test was conducted (since the data is clearly non-normal from the histogram above):

In [11]:
ustat2, pval2 = scipy.stats.mannwhitneyu(first, last, alternative="two-sided")

print("U-statistic: " + str(ustat2))
print("p-value: " + str(pval2))

U-statistic: 83718.0
p-value: 0.324317962648


Since the two sided p-value is lesser than the significance level, the null hypothesis is rejected in favour of the alternative hypothesis that the population means of the first and last 10% of shell weights are statistically different.

### Inferences from Hypothesis Tests

The p-value from the first hypothesis test (which was sorted by rings) was $3.85\times10^{-134}$, which is **very** small, as compared to the p-value of $0.32$ obtained from the comparison of the samples from the second hypothesis test (which was unsorted). From this vast difference, it can be inferred that there is a correlation between the "rings" and "shellweight" variables. However, this inference might not be accurate because the way the results turned out to be might be due to pure chance. Thus, we decided to calculate the Pearson correlation coefficient of the two variables.

This consists of 2 parts:
* Calculating the actual correlation coefficient
* A hypothesis test with the null hypothesis that the 2 variables are correlated and the alternative hypothesis that they are not correlated, with a significance level of $0.05$.

Conducting the test with normalised data:

In [12]:
r, pval = scipy.stats.pearsonr(scipy.stats.zscore(df.shellweight), scipy.stats.zscore(df.rings))
print("Correlation Coefficient: " + str(r))
print("p-value: " + str(pval))

Correlation Coefficient: 0.62757404451
p-value: 0.0


Since the p-value of 0 is lesser than the significance level of $0.05$, the null hypothesis is rejected in favour of the alternative hypothesis that the variables are correlated. This can be further confirmed by the correlation coefficient of $0.628$.

Thus our previous inference from the first 2 hypothesis tests was accurate, and the 2 variables are indeed correlated.

<center><h2>Multivariate Regression</h2></center>

### Multivariate Linear Regression with Gradient Descent

Since the dataset has mutiple variables that can possibly affect the "rings" variable (aka. the age), multivariate linear regression (using gradient descent) was run on the dataset, in order to build a model that would be able to predict the number of rings.

Firstly, a class wrapper was built around the linear regression and gradient descent code to make the process easier:

In [13]:
class Linear_Regression:
    def __init__(self, error_threshold, alpha, normalize):
        self.error_threshold = error_threshold
        self.alpha = alpha
        self.normalize = normalize
        self.theta = None
    
    def predict(self, X, y):
        y = y.values
        if self.normalize:
            X = (X - X.mean()) / X.std()
        X['ones'] = np.ones(len(y))
        X = X.values
        self.theta = np.zeros(X.shape[1])
        cost_history = self.__gradient_descent(X, y)
        predictions = np.dot(X, self.theta)
        return predictions, cost_history
    
    def __compute_cost(self, X, y):
        sum_of_square_errors = np.square(np.dot(X, self.theta) - y).sum()
        cost = sum_of_square_errors / len(y)
        return cost
    
    def __gradient_descent(self, X, y):
        m = len(y)
        cost_history = []
        cost_hist_diff = 1
        while cost_hist_diff > self.error_threshold:
            predictions = np.dot(X, self.theta)
            self.theta = self.theta + ( (self.alpha / m) * np.dot((y - predictions), X) )
            cost_history.append(self.__compute_cost(X, y))
            if len(cost_history) > 1:
                cost_hist_diff = cost_history[-2]-cost_history[-1]
            else:
                cost_hist_diff = 1

        return np.array(cost_history)

Then, linear regression was conducted with a error threshold of $0.0000005$ and a learning rate of $0.1$, with normalized data:

In [14]:
linreg = Linear_Regression(0.0000005, 0.1, True)
X = df.drop("rings", axis=1)
y = df.rings
predictions, costs = linreg.predict(X, y)

The coefficients of the different variables:

In [15]:
X["y-intercept"] = np.ones(len(y))
variables = X.columns
coefs = linreg.theta
varcoefs = pd.DataFrame(columns=["Variables", "Coefficients"], data=list(zip(variables, coefs)))
varcoefs

,Variables,Coefficients
0,length,-0.192604
1,diameter,1.327836
2,height,0.494844
3,wholeweight,4.332848
4,shuckedweight,-4.398463
5,visceraweight,-1.028918
6,shellweight,1.266862
7,y-intercept,9.933684


After which, the $R^2$ value was calculated using the predictions obtained through the linear regression model and the actual values:

In [16]:
r2 = scipy.stats.pearsonr(predictions, y)[0]**2
print("R squared: " + str(r2))

R squared: 0.527595043044


This model gave a $R^2$ value of aprroximately $0.53$. To visually confirm this value, a plot of the actual values against the predictions was made:

In [17]:
scatter = go.Scatter(x=predictions, y=y, opacity=0.7, mode="markers")
layout = go.Layout(title="Plot of Actual Values Against Predictions", xaxis=dict(title="Predictions"), yaxis=dict(title="Actual Values"))
fig = go.Figure(data=[scatter], layout=layout)
plt.iplot(fig)

From the plot, it can be observed that the actual values and predictions are somewhat correlated, thus visualy confirming the $R^2$ value of $0.53$.

The value of the cost function against the number of iterations was then plotted:

In [18]:
scatter = go.Scatter(x=list(range(len(costs))), y=costs,)
layout = go.Layout(title="Plot of Cost Against Iterations", xaxis=dict(title="Iterations"), yaxis=dict(title="Cost"))
fig = go.Figure(data=[scatter], layout=layout)
plt.iplot(fig)

Then a histogram of the residuals was plotted:

In [19]:
hist = [go.Histogram(x=(y-predictions), name="Residuals")]
layout = go.Layout(yaxis={'title': 'Frequency'}, xaxis={'title': 'Residuals'}, title="Histogram of Residuals")
fig = go.Figure(data=hist, layout=layout)
plt.iplot(fig)

As the residuals are mostly centred about $0$ and the histogram is relatively bell shaped, we can infer that the linear regression model is a good fit.

### Mulitvariate Linear Regression with scikit-learn

We did linear regression with our own gradient descent algorithm, but we thought that it would be a good idea to try out the linear regresssion functionality provided by the popular module scikit-learn, as it would probably be more optimised than our algorithm and would probably work better.

First X and y was set up:

In [20]:
X = df.drop("rings", axis=1).values
y = df.rings.values

Then the linear regression classifier was set up:

In [21]:
lr = sklearn.linear_model.LinearRegression(n_jobs=-1, normalize=True)

After that the data was split into training and testing samples randomly, to reduce the chances of overfitting the data:

In [22]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.3, random_state=5)

Then, the classifier was trained and conducted linear regression was conducted:

In [23]:
lr.fit(X_train, y_train)
prediction = lr.predict(X_test)

To benchmark the results with the gradient descent method, $R^{2}$ is calculated:

In [24]:
print("R Squared:", str(lr.score(X_test, y_test).round(2)))

R Squared: 0.48


Plot to visually observe the correlation between predicted values and actual values:

In [25]:
scatter = go.Scatter(x=prediction, y=y_test, opacity=0.7, mode="markers")
layout = go.Layout(title="Plot of Actual Values Against Predictions", xaxis=dict(title="Predictions"), yaxis=dict(title="Actual Values"))
fig3 = go.Figure(data=[scatter], layout=layout)
plt.iplot(fig3)

As you can see from the plot, the predicted values and the actual values do have a somewhat positive corelation, which confirms the $R^2$ value of 0.48. This is lesser than the $R^2$ value of 0.53 as obtained via the gradient descent method (which surprised us quite a bit, since we were expecting a better value using scikit-learn!).

Then a histogram of the residuals was plotted:

In [26]:
hist = [go.Histogram(x=(y_test-prediction), name="Residuals")]
layout = go.Layout(yaxis={'title': 'Frequency'}, xaxis={'title': 'Residuals'}, title="Histogram of Residuals")
fig = go.Figure(data=hist, layout=layout)
plt.iplot(fig)

As the residuals are mostly centred about $0$ and the histogram is relatively bell shaped, we can infer that the linear regression model is a good fit.

### Multivariate Non-Linear Regression

We thought that the $R^2$ value of $0.53$ could be further improved, and thus we started researching on non-linear regression methods. After some research, we found out that non-linear/polynomial regression is simply a special case of linear regression. The figures below explain this.

Lets say we have the variable $x_{1}$, represented as a vector:

$\vec{x_1} = \begin{bmatrix} 1 \\ 2 \\ 3 \\ \end{bmatrix}$

If we were to write the equation that the linear regression model would represent with the corresponding coefficient of $\vec{x_1}$, $\alpha_1$:

$y = \alpha_1 \cdot \begin{bmatrix} 1 \\ 2 \\ 3 \\ \end{bmatrix} = \alpha_1 \cdot \vec{x_1}$

Now, if we square $\vec{x_1}$, we will have:

$\vec{x_1^2} = \begin{bmatrix} 1\\ 4\\ 9\\ \end{bmatrix}$

If we were to write the equation that the linear regression model would represent with the corresponding coefficient of $x_1$ and $x_1^2$, $\alpha_1$ and $\beta_1$ respectively:

$y = \alpha_1 \cdot \begin{bmatrix} 1 \\ 2 \\ 3 \\ \end{bmatrix} + \beta_1 \cdot \begin{bmatrix} 1\\ 4\\ 9\\ \end{bmatrix} = \alpha_1 \cdot \vec{x_1} + \beta_1 \cdot \vec{x_1^2}$

Thus, the model is now quadratic. However, we can treat $x_1$ and $x_1^2$ as separate variables and use a multivariate linear regression model. Thus polynomial/non-linear regresssion is simply a special case of linear regression.

Below, non-linear regression is conducted multiple times using the gradient descent method in order to find the best order of the polynomial:

**Warning: Code takes very long to run**

In [27]:
scores = []
xrange = np.arange(0.05, 10.05, 0.05)
for order in xrange:
    y = df.rings
    X = df.drop("rings", axis=1)
    X2 = np.power(X, order)
    X2.columns = [i+"^"+str(order) for i in X2.columns]
    X2 = X.join(X2)
    linreg = Linear_Regression(0.0000005, 0.1, True)
    predictions, costs = linreg.predict(X2, y)
    scores.append((scipy.stats.pearsonr(predictions, y)[0]**2))

maxscore = max(scores)
best = xrange[scores.index(maxscore)]
minscore = min(scores)
worst = xrange[scores.index(minscore)]

maxscore = round(maxscore, 5)
minscore = round(minscore, 5)

optimised = pd.DataFrame({"order":[best, worst], "r-squared":[maxscore, minscore]})
optimised.head()

,order,r-squared
0,1.8,0.56601
1,1.0,0.52761


In [28]:
scores = [round(i, 5) for i in scores] #to 5sf
line = go.Scatter(x=xrange, y=scores, name="Order of Polynomial")
minmax = go.Scatter(x=optimised.order, y=optimised["r-squared"], mode="markers", marker=dict(color="red"), name="Minimum and Maximum Points")
layout = go.Layout(title="$$\\text{Plot of }R^{2}\\text{Against Order of Polynomial}$$", xaxis=dict(title="Order of Polynomial"), yaxis=dict(title="$$R^{2}$$"))
fig = go.Figure(data=[line, minmax], layout=layout)
plt.iplot(fig)

<center>
Thus the equation that the model represents is
</center>
<h3><center> $y = [\sum\limits_{i=1}^{7}(\alpha_{i}x_{i} + \beta_{i}x_{i}^{1.8})] + c$ </center></h3>
<center>
where $\alpha_{i}$ is the coefficient of $x_{i}$, $\beta_{i}$ is the coefficient of $x_{i}^{1.8}$, and $c$ is the y-intercept.
</center>

<center><h2>Clustering</h2></center>

The data was then clustered using the k-means clustering algorithm, in order to observe if the data formed clusters (that reflected the age of the abalone). This was generally split up into 3 parts:
1. Finding the optimal number of clusters 
2. Clustering the data
3. Visualising the results
4. Predicting age of a new data point using nearest centroid

### 1. Finding the Optimal Number of Clusters

The optimal number of clusters was found by clustering over different number of clusters, finding the distortion/error values and then constructing an elbow plot using those values.

Firstly, the data needed was selected and saved to a new dataframe. The "wholeweight", "shuckedweight" and "diameter" variables were chosen as they have the highest coefficients from the linear regression model, and thus it can be inferred that they affect the clusters more than other variables.  We did not use PCA to reduce the number of variables as we would not be able to identify which variables affect the age since PCA combines the different variables

In [29]:
clustering_data = df[['wholeweight', 'shuckedweight', 'diameter']]

Then the data was normalised (and coverted to a numpy array at the same time):

In [30]:
normdata = scipy.cluster.vq.whiten(clustering_data)

After this, the distortion values for 1 through 29 clusters (29 is the maximum number of rings in this dataset) were calculated and an elbow plot was constucted:

In [31]:
k = 29
dist = []
kvalue = []
for i in range(k):
    kvalue.append(i+1)
    dist.append(scipy.cluster.vq.kmeans(normdata,i+1)[1])
distortion = pd.DataFrame({'distortion' : dist})
distortion.index = kvalue

xvals = distortion.index
yvals = distortion.ix[:,'distortion']

kvalues = [go.Scatter(x=xvals, y=yvals, mode='markers')]
layout = go.Layout(title="Elbow Plot of Distortion Values Against Clusters", xaxis=dict(title="Number of Clusters"), yaxis=dict(title="Distortion Value"))
fig = go.Figure(data=kvalues, layout=layout)
plt.iplot(fig)

From the plot above, the elbow point in the graph is at $k=3$. However to increase the accuracy of our clustering, 5 clusters was chosen as the value of $k$ clustering would be carried out with.

### 2. Clustering the Data

After deciding that $k=5$, the data was clustered.

In [32]:
# Calculate k-means and save the codebook
codebook = scipy.cluster.vq.kmeans(normdata, 5)[0]

# Assign data points to centroids
index = scipy.cluster.vq.vq(normdata,codebook)[0]

### 3. Visualising the Results

The results were then visualised using plot.ly's inbuilt 3D interactive plotting functionality.

In [33]:
clusters = []

layout = go.Layout(showlegend=True, xaxis={'title': 'Shell Weight'}, yaxis={'title': 'Shucked Weight'})

#centroids
clusters.append(go.Scatter3d(x=[i[0] for i in codebook], y=[i[1] for i in codebook], z=[i[2] for i in codebook], name='Centroids', marker={'color': 'red', 'size': 7}, mode='markers'))

#observations
colours = ['orange', 'blue', 'green', 'brown', 'purple']
for i in range(len(codebook)):
    clusters.append(go.Scatter3d(x=normdata[index==i, 0], y=normdata[index==i, 1], z=normdata[index==i, 2], opacity=0.2, name='Cluster ' + str(i+1), marker={'size': 4, 'color': colours[i]}, mode='markers'))

fig = go.Figure(data=clusters, layout=layout)
plt.iplot(fig)

As it can be observed from the plot, the data does not form distinct clusters. It was hypothesised that this is because of the linear nature of age.

### 4. Predicting the Age of a New Data Point Using Nearest Cluster

Although the data did not form distinct clusters, we decided to see if it was possible to predict the age of a certain data point based on the centroid it was assigned to. We decided to do it this way:

1. Split the data up into 90% training and 10% testing data randomly.
2. Calculate the centroids (cluster) using the training data.
3. Calculate the mean and standard deviation of the "rings" variable per cluster.
4. Based on the euclidean distance to the centroids, assign testing data to centroids and calculate mean of "rings" per cluster.
5. Calculate $R^2$ value of the estimated value (mean) of "rings" and the actual value of "rings".

Firstly the data was split up into 90% training and 10% testing data randomly:

In [34]:
X = normdata
y = df.rings.values
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.1, random_state=5)

Then the training data was clustered:

In [35]:
# Calculate k-means and save the codebook
codebook = scipy.cluster.vq.kmeans(X_train, 5)[0]

# Assign data points to centroids
index_train = scipy.cluster.vq.vq(X_train,codebook)[0]

After that, the mean and standard deviation values for the "rings" variable per cluster were calculated.

In [36]:
mean_train = []
std_train = []
for i in range(len(codebook)):
    mean_train.append(np.mean(y_train[index_train==i]))
    std_train.append(np.std(y_train[index_train==i]))
train_stats = pd.DataFrame({"mean": mean_train, "standarddev": std_train})
train_stats

,mean,standarddev
0,8.967991,2.406318
1,6.431616,1.813761
2,10.745659,3.046066
3,11.244029,2.912652
4,12.083333,2.959780


Once that was done, the testing data was assigned to the clusters and the mean value for the "rings" variable per cluster was calculated.

In [37]:
def calculate_distance(d1, d2):
    d = 0
    for i in range(len(d1)):
        d += (d1[i] - d2[i])**2
    d = d**(1/2)
    return d

def return_index(points, centroids):
    index = []
    for i in range(len(points)):
        distances = []
        for j in range(len(centroids)):
            distances.append(calculate_distance(points[i],centroids[j]))
        index.append(distances.index(min(distances)))
    return np.array(index)

index_test = return_index(X_test, codebook)

mean_test = []
std_test = []
for i in range(len(codebook)):
    mean_test.append(np.mean(y_test[index_test==i]))
    std_test.append(np.std(y_test[index_test==i]))
test_stats = pd.DataFrame({"mean": mean_test, "standarddev": std_test})
test_stats

,mean,standarddev
0,9.644444,3.095915
1,6.930556,1.960204
2,10.626087,3.182727
3,11.192308,2.838868
4,11.918919,2.258658


Finally, the $R^2$ value of the means of the training and testing data was calculated:

In [38]:
r2 = scipy.stats.pearsonr(train_stats["mean"].values, test_stats["mean"].values)[0]**2
print("R squared: " + str(round(r2, 3)))

R squared: 0.987


<center><h2>Conclusion</h2></center>
### Strengths of Project
We feel that one of the strengths of our project was that we explored multiple forms of analysis such as hypothesis testing, multiple methods of linear and non-linear regression and clustering. This diversified our analysis, so if one method was unreliable, we still had multiple others to back us up.

We feel that another strength of our project was the data visualisations. We used a service called [plot.ly](http://www.plot.ly) to create our visualisations. It is a much better plotting option than [matplotlib](http://www.matplotlib.org), as it provides minimalist, interactive and easy to configure and use plots out of the box. Our project was also well documented, formatted and explained through the extensive use of markdown and HTML, which makes it easy for the reader to comprehend the project.

### Limitations of Project
One of the obvious limitations of our project was that our data did not form distinct clusters. This severely limited what we could do with the data. Another limitation was that our dataset was relatively small, with only 4177 observations. This limited the accuracy of our analysis.  Also, nonlinear regression takes a long time to run.

### Summary of Analysis
We started off with data wrangling and initial analysis, where we cleaned up that data and plotted a few key variables in order to observe the distribution of the variables. After that, we did hypothesis testing, where we inferred that there is a correlation between the "shellweight" and "rings" variable through the hypothesis tests and confirmed this inference through the Pearson correlation coefficient and hypothesis test.

We then did linear regression using gradient descent and linear regression using scikit-learn, where to our surprise, our own linear regression using gradient descent outperformed scikit-learn's linear regression. We then constructed a non-linear regression model, which managed to pull up the $R^2$ value to a healthy value of $0.57$.

Finally, we clustered the data using the variables "diameter", "shuckedweight", "wholeweight". Although the data did not look like it could be clusted because of the linear nature of age, we did implement a basic version of prediction utilising the clusters, that returned satisfactory results.

### Conclusion of Analysis and Research Question
To reiterate, out research question that we defined at the start of the project was:
**How can the age of abalones be predicted using only physical characteristics and if so, how accurate is the prediction?**

We predicted the age of abalones through regression and clustering. However, regression is not a perfect model, and cannot be used to predict the age very accurately or be used as a substitution to the traditional method of measuring the age of abalones. Clustering is also not a viable solution to predict the age and is also far from a perfect model. Hence, although we did manage to predict the age of abalones using only physical characteristics, the predictions were not very accurate and cannot be substituted for the traditional method of measuring the age of abalones.